Autor: Tatiana Massae Sakata

#  Módulo Spark

## <a name="inicio"> Exercicios Aula 9 </a>

* [Inicio - Parte 1](#inicio)
* [Questão 01](#questao01)
* [Questão 02](#questao02)

In [1]:
from pyspark.sql.functions import *

In [2]:
topic_read = spark.read\
    .format("kafka")\
    .option("kafka.bootstrap.servers","kafka:9092")\
    .option("subscribe","topic-kvspark")\
    .load()

In [3]:
topic_read.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [4]:
topic_string = topic_read.select(col("key"),col("value"))
topic_string.show()

+----+--------------------+
| key|               value|
+----+--------------------+
|null|[31 2C 20 4D 73 6...|
|null|[33 2C 20 4D 73 6...|
|null|[31 2C 20 4D 73 6...|
|null|[33 2C 20 4D 73 6...|
|null|[32 2C 20 4D 73 6...|
|null| [31 2C 4D 73 67 31]|
|null|       [4D 73 67 31]|
|[31]|       [4D 73 67 31]|
|[33]|       [4D 73 67 33]|
|[31]|       [4D 73 67 31]|
|[31]|       [4D 73 67 31]|
|[31]|          [4D 73 67]|
|[33]|       [4D 73 67 33]|
|[34]|       [4D 73 67 34]|
|null| [6D 61 73 73 61 65]|
|null|[32 2C 20 4D 73 6...|
|null|[32 2C 20 4D 73 6...|
|null|[31 2C 20 4D 73 6...|
|null|[33 2C 20 4D 73 6...|
|null| [32 2C 4D 73 67 33]|
+----+--------------------+
only showing top 20 rows



In [5]:
topic_string = topic_read.select(col("key").cast("string"),col("value").cast("string"))
topic_string.show()

+----+-------+
| key|  value|
+----+-------+
|null|1, Msg1|
|null|3, Msg3|
|null|1, Msg1|
|null|3, Msg3|
|null|2, Msg2|
|null| 1,Msg1|
|null|   Msg1|
|   1|   Msg1|
|   3|   Msg3|
|   1|   Msg1|
|   1|   Msg1|
|   1|    Msg|
|   3|   Msg3|
|   4|   Msg4|
|null| massae|
|null|2, Msg2|
|null|2, Msg2|
|null|1, Msg1|
|null|3, Msg3|
|null| 2,Msg3|
+----+-------+
only showing top 20 rows



In [6]:
topic_read_stream = spark.readStream\
    .format("kafka")\
    .option("kafka.bootstrap.servers","kafka:9092")\
    .option("subscribe","topic-kvspark")\
    .load()

In [7]:
topic_read_stream.printSchema()

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)



In [8]:
#Quando for Stream não se consegue visualizar por aqui

topic_string_stream = topic_read_stream.select(col("key").cast("string"),col("value").cast("string"))

In [9]:
topic_string_stream_output = topic_string_stream.writeStream\
    .format("kafka")\
    .option("kafka.bootstrap.servers","kafka:9092")\
    .option("topic","topic-kvspark-output")\
    .option("checkpointLocation","/user/tatiana/kafka_checkpoint")\
    .trigger(continuous="5 second")\
    .start()

In [10]:
topic_string_stream_output.status

{'message': 'Initializing sources',
 'isDataAvailable': False,
 'isTriggerActive': False}

In [11]:
#se verificar no terminal o topico não aparecerá e você pensa que não funcionou
# ele não aparece por que não existe nenhuma informação atual (Stream)
# deverá produzir mensagens em outro terminal

In [12]:
topic_read2 = spark.read\
    .format("kafka")\
    .option("kafka.bootstrap.servers","kafka:9092")\
    .option("subscribe","topic-kvspark-output")\
    .load()
topic_string2 = topic_read2.select(col("key").cast("string"),col("value").cast("string"))
topic_string2.show()

+----+-------+
| key|  value|
+----+-------+
|null|tatiana|
|null| massae|
|null| sakata|
+----+-------+



In [13]:
topic_read_stream = spark.readStream\
    .format("kafka")\
    .option("kafka.bootstrap.servers","kafka:9092")\
    .option("subscribe","topic-kvspark")\
    .option("startingOffsets", "earliest")\
    .load()

In [14]:
topic_string_stream_output = topic_string_stream.writeStream\
    .format("kafka")\
    .option("kafka.bootstrap.servers","kafka:9092")\
    .option("topic","topic-kvspark-")\
    .option("checkpointLocation","/user/tatiana/kafka_checkpoint3")\
    .trigger(continuous="5 second")\
    .start()

In [15]:
topic_string_stream_output.status

{'message': 'Initializing sources',
 'isDataAvailable': False,
 'isTriggerActive': False}

In [17]:
topic_read2 = spark.read\
    .format("kafka")\
    .option("kafka.bootstrap.servers","kafka:9092")\
    .option("subscribe","topic-kvspark")\
    .load()
topic_string2 = topic_read2.select(col("key").cast("string"),col("value").cast("string"),"partition","offset")
topic_string2.show()

+----+-------+---------+------+
| key|  value|partition|offset|
+----+-------+---------+------+
|null|1, Msg1|        1|     0|
|null|3, Msg3|        1|     1|
|null|1, Msg1|        1|     2|
|null|3, Msg3|        1|     3|
|null|2, Msg2|        1|     4|
|null| 1,Msg1|        1|     5|
|null|   Msg1|        1|     6|
|   1|   Msg1|        1|     7|
|   3|   Msg3|        1|     8|
|   1|   Msg1|        1|     9|
|   1|   Msg1|        1|    10|
|   1|    Msg|        1|    11|
|   3|   Msg3|        1|    12|
|   4|   Msg4|        1|    13|
|null| massae|        1|    14|
|null|2, Msg2|        0|     0|
|null|2, Msg2|        0|     1|
|null|1, Msg1|        0|     2|
|null|3, Msg3|        0|     3|
|null| 2,Msg3|        0|     4|
+----+-------+---------+------+
only showing top 20 rows



In [18]:
topic_read2 = spark.read\
    .format("kafka")\
    .option("kafka.bootstrap.servers","kafka:9092")\
    .option("subscribe","topic-kvspark-output")\
    .load()
topic_string2 = topic_read2.select(col("key").cast("string"),col("value").cast("string"),"partition","offset")
topic_string2.show()

+----+-------+---------+------+
| key|  value|partition|offset|
+----+-------+---------+------+
|null|tatiana|        0|     0|
|null| massae|        0|     1|
|null| sakata|        0|     2|
+----+-------+---------+------+



In [19]:
topic_string_stream_output = topic_string_stream.writeStream\
    .format("csv")\
    .option("checkpointLocation","/user/tatiana/kafka_checkpoint4")\
    .option("path","/user/tatiana/kafka/topic-kvspark-output")\
    .start()

In [20]:
!hdfs dfs -ls /user/tatiana/kafka/topic-kvspark-output

Found 2 items
drwxr-xr-x   - root supergroup          0 2022-11-22 00:10 /user/tatiana/kafka/topic-kvspark-output/_spark_metadata
-rw-r--r--   2 root supergroup          0 2022-11-22 00:10 /user/tatiana/kafka/topic-kvspark-output/part-00000-d71456c9-7158-479d-b240-6a4ebd875e1f-c000.csv


<div class="alert alert-block alert-warning">
    IMPORTANTE. 
    Vá na aba “Kernel” e aperte “Shutdown” para encerrar a sessão.
</div>